1. https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea


In [112]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/sec-bert-base")
model = AutoModel.from_pretrained("nlpaueb/sec-bert-base")

Some weights of the model checkpoint at nlpaueb/sec-bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
with open('D:/playdata_mini_proj/data/RSS_for_keybert_remove_trash.pickle', 'rb') as file:
        rss = pickle.load(file)
        
rss.head(2)

,Unnamed: 0,rgs_dt,tck_iem_cd,til_ifo,ctgy_cfc_ifo,mdi_ifo,news_smy_ifo,rld_ose_iem_tck_cd,url_ifo
0,0,20230102,NGS,Hoak & Co. Urges NGS Board to Halt CEO Search ...,Investing,Fintel,"Fintel reports that Hoak Public Equities, LP, ...",NGS,https://www.nasdaq.com/articles/hoak-co.-urges...
1,1,20230102,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,PhenixFIN has been beaten down lately with to...,PFX,https://www.nasdaq.com/articles/heres-why-phen...


In [85]:
rss['news_smy_ifo'][2]

"There's a disconnect setting up in the energy market that's a flashing yellow light for investors--particularly if you've been playing high-flying oil stocks in ."

In [125]:
# 입력 텍스트 준비
text = rss['news_smy_ifo'][3]

pattern = r'[.,"\'!@#$%^&*()_+={}\[\]:;<>?|\\`~-]'
    
# 정규식 패턴과 일치하는 부분을 빈 문자열로 대체
text = re.sub(pattern, '', text)

# 입력 텍스트 토큰화 및 모델 입력 형식으로 변환
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# 모델을 사용하여 예측 수행
outputs = model(**inputs)

In [119]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3294,  0.1290,  0.0040,  ..., -0.3903, -0.2061, -0.1667],
         [ 0.0619,  0.1521,  0.0187,  ..., -0.4824,  0.2735, -0.0074],
         [ 0.2557,  0.7708, -0.4982,  ..., -0.3711,  0.3549,  0.1538],
         ...,
         [-0.1572,  0.1325,  0.1423,  ..., -0.3493,  0.5477, -0.1486],
         [ 0.6371,  1.2239,  0.2806,  ..., -0.0706,  0.3074,  0.0390],
         [ 0.6367,  1.2295,  0.2819,  ..., -0.0705,  0.3059,  0.0394]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-6.2986e-02,  5.0106e-02,  8.0713e-01, -7.3770e-02,  1.6770e-01,
          5.8577e-02, -9.4330e-02, -9.4195e-02, -2.6270e-02,  2.7925e-01,
          3.4131e-01, -5.8779e-01, -2.2108e-02, -1.2609e-01,  9.6673e-01,
         -9.9910e-01,  2.1456e-01,  1.0556e-01, -7.7126e-01, -1.2521e-02,
         -8.7439e-02,  1.4069e-01,  2.8582e-01,  9.9399e-01,  1.4040e-01,
          2.3434e-01,  9.0676e-01,  1.8533e-01, -6.3667e-02, -8.288

#### BaseModelOutputWithPoolingAndCrossAttentions:

- 모델의 마지막 레이어의 히든 상태, 풀링된 히든 상태, 그리고 교차 어텐션(마스크 어텐션)에 관련된 정보
- 모델의 텍스트 인코딩, 문맥 인코딩 등을 포함하고 있으며, 이를 사용하여 다양한 NLP 태스크를 수행할 수 있습니다.

#### NativeLayerNormBackward:
- PyTorch나 다른 딥러닝 프레임워크에서 내부적으로 사용되는 함수나 클래스
- Layer Normalization과 관련된 역전파 계산을 담당하는 함수로, 모델의 학습 과정에서 그래디언트 업데이트 및 가중치 조정에 필요

#### - doc2vec  또는 키워드 추출에는 BaseModelOutputWithPoolingAndCrossAttentions를 사용해야 한다

In [ ]:
tokenizer.convert_tokens_to_string(np.sort(np.concatenate(np.concatenate(outputs[0].detach().numpy(), axis=0)))[-5:])

In [105]:
top_indices

tensor([321, 115, 413, 614, 762])

In [106]:
attention_weights

tensor([ 1.2250e-01,  2.0185e-01,  1.7606e-01, -1.2196e-01,  1.8739e-01,
        -1.4897e-01, -1.0231e-01, -3.8283e-01,  2.0890e-01,  2.8797e-01,
         1.9183e-01, -2.9690e-01, -1.2003e-02,  2.1363e-01,  6.3982e-02,
        -5.3046e-02,  1.5918e-01, -2.3263e-01,  1.0313e-01, -2.0476e-01,
        -5.3746e-02, -2.4057e-02, -3.4127e-01,  5.7323e-02, -1.8862e-01,
         2.2916e-02, -1.8096e-01, -1.0656e-01,  1.5389e-02,  1.4621e-02,
         2.7918e-01, -3.2312e-03, -1.9995e-01, -7.7912e-02, -2.6439e-01,
        -9.8828e-02, -1.8667e-02, -7.2085e-02,  8.0069e-02, -2.4174e-01,
        -1.5325e-01, -1.5232e-01,  1.9089e-01,  4.5927e-02,  3.7221e-02,
         4.4159e-01,  3.7878e-01,  1.6025e-01,  2.3588e-01, -4.7647e-01,
        -1.0501e-01, -2.5837e-01, -1.2175e-01,  6.9481e-02, -5.7367e-02,
        -1.6792e-01,  1.6490e-01,  1.0767e-01,  1.2547e-01,  6.9212e-02,
         2.3671e-01,  9.2116e-02,  5.2051e-01,  3.4370e-01,  3.1913e-01,
        -1.9623e-02, -2.8978e-01, -1.9267e-01, -6.4

In [126]:
outputs[0][0].mean(dim=1).shape

torch.Size([44])

In [124]:
import nltk
from nltk import word_tokenize, pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

## 입력 텍스트 준비
text = rss['news_smy_ifo'][3]
print(text)

pattern = r'[.,"\'!@#$%^&*()_+={}\[\]:;<>?|\\`~-]'
    
# 정규식 패턴과 일치하는 부분을 빈 문자열로 대체
text = re.sub(pattern, '', text)

# 문장 토큰화
tokens = word_tokenize(text)

# 품사 태깅
tagged_words = pos_tag(tokens)

# 명사만 추출
nouns = [word for word, tag in tagged_words if tag.startswith('N')]

# 결과 출력
print("명사:", nouns)

sentences = ' '.join(nouns)

# 입력 텍스트 토큰화 및 모델 입력 형식으로 변환
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

# 모델을 사용하여 예측 수행
outputs = model(**inputs)

# 가장 중요한 단어 토큰 인덱스 추출 (특수 토큰 및 '[SEP]' 제외)
# 어텐션 가중치 평균 계산
attention_weights = outputs[0][0].mean(dim=1)

# 가장 중요한 토큰 인덱스 추출
top_indices = attention_weights.topk(5).indices

# 가장 중요한 토큰과 해당 가중치 출력
for idx in top_indices:
    token = tokenizer.convert_ids_to_tokens(inputs.input_ids[0][idx].item())
    weight = attention_weights[idx].item()
    print(f"토큰: {token}, 가중치: {weight}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Malteries Soufflet, a branch of French agribusiness group InVivo, said on Monday it had signed an agreement to take over Belgian malthouse Castle Malting, one of the oldest producers of the beer ingredient in the world.
명사: ['Malteries', 'Soufflet', 'branch', 'agribusiness', 'group', 'InVivo', 'Monday', 'agreement', 'malthouse', 'Castle', 'Malting', 'producers', 'beer', 'ingredient', 'world']
토큰: [SEP], 가중치: 0.004538911394774914
토큰: ##ing, 가중치: 6.40129073872231e-05
토큰: ##ff, 가중치: -0.00026278631412424147
토큰: malt, 가중치: -0.00028538014157675207
토큰: malt, 가중치: -0.00029104939312674105


In [96]:
inputs.input_ids.shape

torch.Size([1, 40])